# 파이썬 모델링 라이브러리
Patsy에 중점적으로 하도록 하겠습니다.

## pandas와 모델 코드의 인터페이스

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
import pandas as pd
import numpy as np

data = pd.DataFrame(
    {
        "x0": [1, 2, 3, 4, 5],
        "x1": [0.01, -0.01, 0.25, -4.1, 0.0],
        "y": [-1.5, 0.0, 3.6, 1.3, -2.0],
    }
)
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [11]:
print(type(data.values), "\n")
data.values  # 넘파이 값으로 받기

<class 'numpy.ndarray'> 



array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [9]:
df3 = data.copy()
df3["strings"] = ["a", "b", "c", "d", "e"]
df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [12]:
print(type(df3.values), "\n")
df3.values  # 데이터 값들이 다르면 ndarray 객체 object로

<class 'numpy.ndarray'> 



array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

In [13]:
data["category"] = pd.Categorical(["a", "b", "a", "a", "b"], categories=["a", "b"])
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [14]:
dummies = pd.get_dummies(data.category, prefix="category")
data_with_dummies = data.drop("category", axis=1).join(dummies)
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


## Patsy를 이용해서 모델 생성하기

통계 모델(특히 선형 모델)을 위한 파이썬 라이브러리이며 R이나 S 통계프로그래밍 언어에서 사용하는

수식 문법과 비슷한 형식의 문자열 기반 '수식 문법'을 제공함.

> R은 S를 기반으로한 통계학 전공자들이 많이 쓰는 프로그래밍 언어입니당.                    
> https://cran.r-project.org/bin/windows/base/ 여기서 다운로드 가능합니다.(윈도우)


y ~ x0 + x1

이때 x0 + x1은 단순히 더하는게 아니라 배열을 설계하는 용법임.

In [18]:
data = pd.DataFrame(
    {
        "x0": [1, 2, 3, 4, 5],
        "x1": [0.01, -0.01, 0.25, -4.1, 0.0],
        "y": [-1.5, 0.0, 3.6, 1.3, -2.0],
    }
)
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [19]:
import patsy

y, X = patsy.dmatrices("y ~ x0 + x1", data)

In [20]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [21]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [22]:
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [23]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

In [24]:
patsy.dmatrices("y ~ x0 + x1 + 0", data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

In [25]:
coef, resid, _, _ = np.linalg.lstsq(X, y)

C:\Users\WJ\anaconda3\envs\aiffel\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


In [26]:
coef

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [27]:
coef = pd.Series(coef.squeeze(), index=X.design_info.column_names)
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

### Patsy 용법으로 데이터 변환하기

In [28]:
y, X = patsy.dmatrices("y ~ x0 + np.log(np.abs(x1) + 1)", data)
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

In [29]:
y, X = patsy.dmatrices("y ~ standardize(x0) + center(x1)", data) #standardize() -> 평균 0 , 분산 1, center(평균을 빼서 평균 0으로)
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

모델을 어떤 데이터셋에 피팅하고 그 다음에 다른 모델에 기반하여 평가하는 경우

이는 홀드-아웃이거나 신규 데이터가 나중에 관측되는 경우임. 

이때 centering이나 standardize는 새로운 데이터에 기반하여 예측하기 위한 용도로 모델을 사용한다면 주의해야함.

이를 상태를 가지는 stateful 변환이라고 하고, 새로운 데이터셋을 변경하기 위해 

원본 데이터의 표준편차나 평균 같은 통계를 사용해야하기 때문

In [30]:
new_data = pd.DataFrame(
    {"x0": [6, 7, 8, 9], "x1": [3.1, -0.5, 0, 2.3], "y": [1, 2, 3, 4]}
)
new_X = patsy.build_design_matrices([X.design_info], new_data)
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

Patsy 문법에 + 기호는 덧셈이 아니라 데이터셋에서 이름으로 컬럼을 추가하고 싶다면 I로 둘러싸야함

In [31]:
y, X = patsy.dmatrices("y ~ I(x0 + x1)", data)
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

### 범주형 데이터와 Patsy

In [32]:
data = pd.DataFrame(
    {
        "key1": ["a", "a", "b", "b", "a", "b", "a", "b"],
        "key2": [0, 1, 0, 1, 0, 1, 0, 0],
        "v1": [1, 2, 3, 4, 5, 6, 7, 8],
        "v2": [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7],
    }
)
y, X = patsy.dmatrices("v2 ~ key1", data)
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

모델에서 intercept를 생략하면 각 범주값의 컬럼은 모델 설계 배열에 포함됨

In [33]:
y, X = patsy.dmatrices("v2 ~ key1 + 0", data)
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

C를 싸주면 범주형으로 해석

In [34]:
y, X = patsy.dmatrices("v2 ~ C(key2)", data)
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

ANOVA(분석) 모델에서처럼 key1:key2를 사용할 수 있게 됨

In [35]:
data["key2"] = data["key2"].map({0: "zero", 1: "one"})
data
y, X = patsy.dmatrices("v2 ~ key1 + key2", data)
X
y, X = patsy.dmatrices("v2 ~ key1 + key2 + key1:key2", data)
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)